# Requirements

In [1]:
! pip install transformers torch sentencepiece datasets accelerate evaluate sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 2.3 MB/s eta 0:00:00
  Attempting unin

# Dataset

In [2]:
from datasets import load_dataset
import torch

with torch.cuda.device(0):
  label2id = {
      'negative': 0,
      'neutral': 1,
      'positive': 2,
  }

  id2label = {
      0: 'negative',
      1: 'neutral',
      2: 'positive',
  }

  loaded_data = load_dataset('arize-ai/beer_reviews_label_drift_neg')

  zero_shot_test, test_expected = [], []
  prompt = ' Is this a positive, neutral or negative review?'

  for data in loaded_data['production']:
    zero_shot_test.append(data['text'] + prompt)
    test_expected.append(int(data['label']))
  
  zero_shot_valid, valid_expected = [], []

  for data in loaded_data['validation']:
    zero_shot_valid.append(data['text'] + prompt)
    valid_expected.append(int(data['label']))

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating training split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating production split: 0 examples [00:00, ? examples/s]

Dataset beer_reviews_label_drift_neg downloaded and prepared to /root/.cache/huggingface/datasets/arize-ai___beer_reviews_label_drift_neg/default/1.0.0/19f47acda7a949b5528784d68d9f6aa4eaa144ba8ecaf7a234e4243e2ad1c4a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Training

In [ ]:
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/t5/data/train.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/t5/run_translation.py'

--2023-02-14 10:58:57--  https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/t5/data/train.json
Resolving git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)... 150.254.78.40
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6403138 (6.1M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]   6.11M  3.50MB/s    in 1.7s    

2023-02-14 10:59:00 (3.50 MB/s) - ‘train.json’ saved [6403138/6403138]

--2023-02-14 10:59:00--  https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/t5/run_translation.py
Resolving git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)... 150.254.78.40
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29986 (29K) [text/plain]
Saving to: ‘run_translation.py’

run_translation.py  100%[===================>]  29.28K  --.-KB/s    in 0.1s    

2023-02-14 10:59:01 (2

In [ ]:
! python run_translation.py \
  --cache_dir .cache_training \
  --model_name_or_path google/flan-t5-base \
  --train_file train.json \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --source_lang "text" \
  --target_lang "label" \
  --source_prefix "classification" \
  --max_source_length 256 \
  --max_target_length 128 \
  --generation_max_length 128 \
  --do_train \
  --predict_with_generate \
  --num_train_epochs 1 \
  --save_strategy steps \
  --save_steps 250 \
  --save_total_limit 5 \
  --logging_strategy steps \
  --logging_steps 100 \
  --greater_is_better True \
  --output_dir drive/MyDrive/out/flan-t5

2023-02-14 11:15:17.719454: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 11:15:17.719736: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 11:15:17.719757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
INFO:__main__:Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch

# Pipeline

In [4]:
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/all_results.json' -O 'flan-t5/all_results.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/config.json' -O 'flan-t5/config.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/generation_config.json' -O 'flan-t5/generation_config.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/pytorch_model.bin' -O 'flan-t5/pytorch_model.bin'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/special_tokens_map.json' -O 'flan-t5/special_tokens_map.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/spiece.model' -O 'flan-t5/spiece.model'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/tokenizer_config.json' -O 'flan-t5/tokenizer_config.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/tokenizer.json' -O 'flan-t5/tokenizer.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/train_results.json' -O 'flan-t5/train_results.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/trainer_state.json' -O 'flan-t5/trainer_state.json'
! wget 'https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/training_args.bin' -O 'flan-t5/training_args.bin'


--2023-02-14 12:07:17--  https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/all_results.json
Resolving git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)... 150.254.78.40
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193 [text/plain]
Saving to: ‘flan-t5/all_results.json’

flan-t5/all_results 100%[===================>]     193  --.-KB/s    in 0s      

2023-02-14 12:07:18 (43.6 MB/s) - ‘flan-t5/all_results.json’ saved [193/193]

--2023-02-14 12:07:18--  https://git.wmi.amu.edu.pl/s443930/deep-nlp/raw/branch/master/flan-t5/out/config.json
Resolving git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)... 150.254.78.40
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1528 (1.5K) [text/plain]
Saving to: ‘flan-t5/config.json’

flan-t5/config.json 100%[===================>]   1.49K  --.-KB/s    in 0s      



In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM, AutoModelForSeq2SeqLM
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [6]:
device = torch.device('cuda:0')

model_name = 'flan-t5'

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    low_cpu_mem_usage=True, 
    torch_dtype=torch.float32
  ).to(device)

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    device=device
  )

In [7]:
classifier = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=device)

In [9]:
with torch.cuda.device(0):
  eval_predicted = []

  for review in tqdm(zero_shot_valid):
    eval_predicted.append(classifier(review, max_length=60)[0]['generated_text'])

  0%|          | 0/1260 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1260/1260 [01:21<00:00, 15.39it/s]


In [10]:
import numpy as np 

np.unique(eval_predicted)

array(['0', '1', '2', 'negative', 'positive'], dtype='<U8')

In [11]:
eval_predicted_labels = [
    0 if p == 'negative'
    else 2 if p == 'positive'
    else int(p)
    for p in eval_predicted
]

print(f'eval accuracy: {accuracy_score(valid_expected, eval_predicted_labels)}')

eval accuracy: 0.7611111111111111


In [13]:
with open('eval_predicted_labels.txt', 'w') as fp:
    fp.write("\n".join(str(item) for item in eval_predicted_labels))

In [14]:
with torch.cuda.device(0):
  test_predicted = []

  for review in tqdm(zero_shot_test):
    test_predicted.append(classifier(review, max_length=60)[0]['generated_text'])

  0%|          | 0/27742 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 27742/27742 [26:00<00:00, 17.78it/s]


In [15]:
np.unique(test_predicted)

array(['0', '1', '2', 'negative'], dtype='<U8')

In [16]:
test_predicted_labels = [
    0 if p == 'negative'
    else int(p) 
    for p in test_predicted
    ]

print(f'test accuracy: {accuracy_score(test_expected, test_predicted_labels)}')

test accuracy: 0.7269483094225362


In [18]:
with open('test_predicted_labels.txt', 'w') as fp:
    fp.write("\n".join(str(item) for item in test_predicted_labels))